## **Weekly Labeling and Trading Strategy Evaluation for Boeing (BA)**

### **Introduction**

My trading strategy is based on weekly price stability.  
I will only remain **invested in the market** if I expect the **next Friday’s closing price** to stay within **±$10** of this Friday’s close.  
If the expected movement exceeds this range, I will **move to cash** to avoid exposure to volatility.

Using this rule, I manually labeled each week as **green** (stay invested) or **red** (move to cash).  
These manually assigned labels will later serve as the **training data** for a machine learning model that predicts whether to be in the market or in cash in future weeks.


### **Trading Strategy Rules**

I start with **$100** prior to Week 1. The trading strategy is implemented as follows:

1. **First Green Week**  
   - For the very first *green* week, invest **$100** by buying (possibly fractional) shares of the stock at the **opening price** of the first trading day of that week.  

2. **If the Next Week Is Red**  
   - (a) If you currently hold a position (this week is *green*), **sell all shares** at the **closing price** of the last trading day of this week.  
   - (b) If you have no position (this week is *red*), do nothing and **remain in cash** for the next week.

3. **If the Next Week Is Green**  
   - (a) If you already have a position (this week is *green*), **stay invested** for the next week.  
   - (b) If you have no position (this week is *red*), **buy shares** at the **opening price of the next week**, investing all available cash.

4. **Assumptions**  
   - Ignore trading costs and assume the ability to buy or sell at **open** or **(adjusted) closing** prices without slippage.


In [9]:
import pandas as pd
import numpy as np

In [10]:
ba_df = pd.read_csv('ba_weekly_return_detailed.csv')
ba_df.head()

,Close,High,Low,Open,Volume,Return,Date,Week_Number,Year,Day,Weekday,mean_return,volatility,label
0,331.348572,331.378393,325.761816,326.606765,4544400,0.000,1/2/20,0,2020,2,Thursday,-0.0840,0.118794,green
1,330.791901,332.909308,328.346429,328.674494,3875900,-0.168,1/3/20,0,2020,3,Friday,-0.0840,0.118794,green
2,331.766083,332.879454,325.940756,327.352341,5355000,0.294,1/6/20,1,2020,6,Monday,-0.1612,1.584772,green
3,335.285156,342.154291,328.754007,332.283029,9898600,1.061,1/7/20,1,2020,7,Tuesday,-0.1612,1.584772,green
4,329.410095,332.054366,327.650575,330.434002,8239200,-1.752,1/8/20,1,2020,8,Wednesday,-0.1612,1.584772,green


In [11]:
ba_df.groupby('label')['Open'].count().reset_index()

,label,Open
0,green,1020
1,red,232


After manually assigning labels to each week, I obtained 1020 green and 232 red weeks. This shows that the strategy keeps me in the market most of the time, while only moving to cash during periods of high volatility or sharp price changes.

In [12]:
ba_df = ba_df.sort_values(['Year', 'Week_Number', 'Day']).reset_index(drop=True)
# print( (100 / ba_df.iloc[0]['Open']) * ba_df.iloc[-1]['Close'] )

ba_yearly = ba_df.groupby(["Year"]).agg(
    Close_y = ('Close', 'last')
).reset_index().sort_values(['Year']).reset_index(drop=True)


shares = (100 / ba_df.iloc[0]['Open'])
portfolio=ba_yearly.copy()
portfolio["BuyHold"] = round(shares * ba_yearly['Close_y'], 2)

print(portfolio)
    

   Year     Close_y  BuyHold
0  2020  214.059998    65.54
1  2021  201.320007    61.64
2  2022  190.490005    58.32
3  2023  260.660004    79.81
4  2024  176.550003    54.06


To calculate earning/loss using buy-and-hold strategy, I invested $100 at the opening price of the first trading day in 2020. I held the same number of shares continuously for five years, and sold them at the closing price of the last trading day in 2024.
As a result, the portfolio declined from $100 to $54.06 in 5 years.

In [13]:
ba_weekly_labels = ba_df.groupby(["Year", "Week_Number"]).agg(
    Open_w = ('Open', 'first'),
    Close_w = ('Close', 'last'),
    label = ('label', 'first')
).reset_index().sort_values(['Year','Week_Number']).reset_index(drop=True)

# print(ba_weekly_labels.head(10))

cash = 100
shares = 0
start_invested = False
results = {}


for i in range(len(ba_weekly_labels)-1):
    week = ba_weekly_labels.iloc[i]
    next_week = ba_weekly_labels.iloc[i+1]
    
    if(shares == 0):
        if((not start_invested) and week['label'] == "green"): #very first "green" week 
            shares = cash / week['Open_w']
            cash = 0
            start_invested = True
        elif(next_week['label'] == "green"):
            shares = cash / next_week['Open_w']
            cash = 0
    elif (shares > 0 and next_week['label'] == "red"):
        cash = shares * week['Close_w']
        shares = 0
        
    if week['Week_Number'] == 52: #store yearly earnings/losses
        yr_total = shares*week['Close_w'] if shares > 0 else cash
        results[week['Year']] = (yr_total)
        
# for last year
final_data = ba_weekly_labels.iloc[-1]
if(shares > 0):
    cash = shares * final_data['Close_w']
    shares = 0

results[final_data['Year']] = (cash)
portfolio["Labeling"] = round(portfolio["Year"].map(results), 2)

print(round(cash, 2))
# print(shares)
# print(final_close)

6529.31


In [14]:
print(portfolio)

   Year     Close_y  BuyHold  Labeling
0  2020  214.059998    65.54    546.20
1  2021  201.320007    61.64    852.86
2  2022  190.490005    58.32   2404.52
3  2023  260.660004    79.81   4115.67
4  2024  176.550003    54.06   6529.31


The label-based strategy grew to $6529.31 over the 2020-2024 period. Since my strategy avoids downtrends and unstable weekly movements, probably that's why I got this large difference compared to buy-and-hold strategy. 

The benefit was particularly clear in 2022–2023, when Boeing experienced significant recoveries. During these years, the label-based approach captured the gains while buy-and-hold remained deeply underwater.